In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Average
from keras.layers import Activation
from keras.layers import GlobalAveragePooling2D

from tensorflow.keras import layers
print("TF version", tf.__version__)

In [ ]:
from sklearn.model_selection import train_test_split
KMNIST_WIDTH, KMNIST_HEIGHT = 28, 28
def load_data(do_reshape=False, resize_shape=None, channel_num=1):
    DMNIST_pd = pd.read_csv("/kaggle/input/Kannada-MNIST/Dig-MNIST.csv")
    train_pd = pd.read_csv("/kaggle/input/Kannada-MNIST/train.csv")
    test_pd = pd.read_csv("/kaggle/input/Kannada-MNIST/test.csv")
    x_train_np, y_train_np = train_pd.iloc[:,1:].to_numpy(), train_pd.iloc[:,0].to_numpy()
    x_DMNIST_np, y_DMNIST_np = DMNIST_pd.iloc[:,1:].to_numpy(), DMNIST_pd.iloc[:,0].to_numpy()
    
    x_data=np.concatenate((x_train_np,x_DMNIST_np))
    y_data=np.concatenate((y_train_np,y_DMNIST_np))
    x_test_full = test_pd.iloc[:,1:].to_numpy()
    
    x_train_full, x_valid_full, y_train_full, y_valid_full = train_test_split(x_data, y_data,test_size=0.2, random_state=42)
    x_train_full = x_train_full.astype('float32')
    x_valid_full = x_valid_full.astype('float32')
    x_test_full = x_test_full.astype('float32')
    
    # resize image for different NN
    if resize_shape:
        x_train_full = tf_img_resize(x_train_full, resize_shape)
        x_valid_full = tf_img_resize(x_valid_full, resize_shape)
        x_test_full = tf_img_resize(x_test_full, resize_shape)
    # reshape to 2D for CNN
    if do_reshape:
        (w, h) = (KMNIST_WIDTH, KMNIST_HEIGHT) if not resize_shape else (resize_shape[0], resize_shape[1])
        x_train_full = np.reshape(x_train_full, (-1, w, h, 1)).astype('float32')
        x_valid_full = np.reshape(x_valid_full, (-1, w, h, 1)).astype('float32')
        x_test_full = np.reshape(x_test_full, (-1, w, h, 1)).astype('float32')
    if channel_num > 1:
        x_train_full = np.repeat(x_train_full, channel_num, -1)
        x_valid_full = np.repeat(x_valid_full, channel_num, -1)
        x_test_full = np.repeat(x_test_full, channel_num, -1)
        
    return (x_train_full, y_train_full), (x_valid_full, y_valid_full), x_test_full

import cv2
def cv_img_resize(imgs, resize_shape):
    def my_cv_resize(a):
        return cv2.resize(a.reshape(28,28), resize_shape, interpolation=cv2.INTER_CUBIC)
        #return tf.image.resize(a.reshape(28,28), (92,92), method="bicubic")
    return np.apply_along_axis(my_cv_resize, 1, imgs)
    #return np.apply_along_axis(tf.image.resize, 0, imgs, resize, method="bicubic")

def tf_img_resize(imgs, resize_shape):
    def my_cv_resize(a):
        #return cv2.resize(a.reshape(28,28), resize_shape, interpolation=cv2.INTER_CUBIC)
        return tf.image.resize(a.reshape(28,28,1), [96,96], method="bicubic")
    return np.apply_along_axis(my_cv_resize, 1, imgs)
    #return np.apply_along_axis(tf.image.resize, 0, imgs, resize, method="bicubic")



def show_imgs(x_train, y_train, rows=4):
    classes = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    print(classes)
    for y, cls in enumerate(classes):
        idxs = np.nonzero([i == y for i in y_train])
        idxs = np.random.choice(idxs[0], rows)
        for i , idx in enumerate(idxs):
            plt_idx = i * len(classes) + y + 1
            plt.subplot(rows, len(classes), plt_idx)
            plt.imshow(x_train[idx].reshape((x_train.shape[1], x_train.shape[2],x_train.shape[3])))
            plt.axis("off")
            if i == 0:
                plt.title(cls)
    plt.show()



### Data Loading & Data Preprocessing


In [ ]:
(x_train, y_train), (x_valid, y_valid), x_test = load_data(do_reshape=True, resize_shape=(96,96), channel_num=3)

print("Train shape", x_train.shape, y_train.shape)
print("Valid shape", x_valid.shape, y_valid.shape)
print("Test shape", x_test.shape)
show_imgs(x_train, y_train, rows=4)

# Model Creation:CNN

Simple CNN based score: 0.968

Improvements:
- Data Augmentation +0.01, 0.978
- Not zero center +0.003, 0.981
- LearningRateScheduler callbacks + 0.003, 0.983
- More Dropout +0.002, 0.985
- Average layers +0.003, 0.988
- Data Augmentation Enhancement +0.002, 0.99
- 1x1 kernel +0.001, 0.9916

## Implementation 1: Module - Class Instance

In [ ]:
class average_CNN(tf.keras.Model):
    def __init__(self, feature_maps = 32, kernel_sizes = (1,3,5), **kwargs):
        super(average_CNN, self).__init__(**kwargs)
        self.cnn_layers = []
        for kernel_size in kernel_sizes:
            self.cnn_layers.append(
                [Conv2D(feature_maps, kernel_size, activation="relu", kernel_initializer="he_uniform", padding="same"),
                Conv2D(feature_maps, kernel_size, activation="relu", kernel_initializer="he_uniform", padding="same"),
                BatchNormalization()
                ]
            )
        self.avg_layer = Average()
        self.act = Activation('relu')
        self.bn4 = BatchNormalization()
        self.pool = MaxPool2D()
        self.drop = Dropout(0.2)
        
    def call(self, inputs):
        x = inputs
        self.middle_outs = []
        for layer_set in self.cnn_layers:
            for layer in layer_set:
                x = layer(x)
            self.middle_outs.append(x)
            
        x = self.avg_layer(self.middle_outs)
        x = self.act(x)
        x = self.bn4(x)
        x = self.pool(x)
        x = self.drop(x)
        return x

class my_CNN(tf.keras.Model):
    def __init__(self, **kwargs):
        super(my_CNN, self).__init__(**kwargs)
        self.avg_c1 = average_CNN( 32, (1,3,5))
        self.avg_c2 = average_CNN( 64, (1,3,5))
        self.avg_c3 = average_CNN(128, (1,3,5))
        
        self.flat = Flatten()
        self.den1 = Dense(128, activation="relu", kernel_initializer="he_uniform")
        self.drop = Dropout(0.2)
        self.den2 = Dense(10, activation="softmax")
        
    def call(self, inputs):
        x = self.avg_c1(inputs)
        x = self.avg_c2(x)
        x = self.avg_c3(x)
        
        x = self.flat(x)
        x = self.den1(x)
        x = self.drop(x)
        x = self.den2(x)
        
        return x
        

In [ ]:
def module_CNN():
    my_cnn = my_CNN()
    my_cnn.compile(loss="sparse_categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
    return my_cnn

## Implementation 2: Functional API

In [ ]:
def model_CNN():
    model_input = keras.Input(shape=(28,28,1))
    
    C1_1 = Conv2D(32, 1, activation="relu", kernel_initializer="he_uniform", padding="same")(model_input)
    C1_2 = Conv2D(32, 1, activation="relu", kernel_initializer="he_uniform", padding="same")(C1_1)
    C1_3 = BatchNormalization()(C1_2)
    C2_1 = Conv2D(32, 3, activation="relu", kernel_initializer="he_uniform", padding="same")(model_input)
    C2_2 = Conv2D(32, 3, activation="relu", kernel_initializer="he_uniform", padding="same")(C2_1)
    C2_3 = BatchNormalization()(C2_2)
    C3_1 = Conv2D(32, 5, activation="relu", kernel_initializer="he_uniform", padding="same")(model_input)
    C3_2 = Conv2D(32, 5, activation="relu", kernel_initializer="he_uniform", padding="same")(C3_1)
    C3_3 = BatchNormalization()(C3_2)

    avg1_1 = Average()([C1_3, C2_3, C3_3])
    avg1_2 = Activation('relu')(avg1_1)
    avg1_3 = BatchNormalization()(avg1_2)
    mp1 = MaxPool2D()(avg1_3)
    out1 = Dropout(0.2)(mp1)
    # average1 = average_CNN(feature_maps= 32, kernels = (1, 3, 5))
    C1_1 = Conv2D(64, 1, activation="relu", kernel_initializer="he_uniform", padding="same")(out1)
    C1_2 = Conv2D(64, 1, activation="relu", kernel_initializer="he_uniform", padding="same")(C1_1)
    C1_3 = BatchNormalization()(C1_2)
    C2_1 = Conv2D(64, 3, activation="relu", kernel_initializer="he_uniform", padding="same")(out1)
    C2_2 = Conv2D(64, 3, activation="relu", kernel_initializer="he_uniform", padding="same")(C2_1)
    C2_3 = BatchNormalization()(C2_2)
    C3_1 = Conv2D(64, 5, activation="relu", kernel_initializer="he_uniform", padding="same")(out1)
    C3_2 = Conv2D(64, 5, activation="relu", kernel_initializer="he_uniform", padding="same")(C3_1)
    C3_3 = BatchNormalization()(C3_2)

    avg2_1 = Average()([C1_3, C2_3, C3_3])
    avg2_2 = Activation('relu')(avg2_1)
    avg2_3 = BatchNormalization()(avg2_2)
    mp2 = MaxPool2D()(avg2_3)
    out2 = Dropout(0.2)(mp2)
    
    C1_1 = Conv2D(128, 1, activation="relu", kernel_initializer="he_uniform", padding="same")(out2)
    C1_2 = Conv2D(128, 1, activation="relu", kernel_initializer="he_uniform", padding="same")(C1_1)
    C1_3 = BatchNormalization()(C1_2)
    C2_1 = Conv2D(128, 3, activation="relu", kernel_initializer="he_uniform", padding="same")(out2)
    C2_2 = Conv2D(128, 3, activation="relu", kernel_initializer="he_uniform", padding="same")(C2_1)
    C2_3 = BatchNormalization()(C2_2)
    C3_1 = Conv2D(128, 5, activation="relu", kernel_initializer="he_uniform", padding="same")(out2)
    C3_2 = Conv2D(128, 5, activation="relu", kernel_initializer="he_uniform", padding="same")(C3_1)
    C3_3 = BatchNormalization()(C3_2)

    avg3_1 = Average()([C1_3, C2_3, C3_3])
    avg3_2 = Activation('relu')(avg3_1)
    avg3_3 = BatchNormalization()(avg3_2)
    mp3 = MaxPool2D()(avg3_3)
    out3 = Dropout(0.2)(mp3)
    
    out = Flatten()(out3)
    out = Dense(128, activation="relu", kernel_initializer="he_uniform")(out)
    out = Dropout(0.2)(out)
    out = Dense(10, activation="softmax")(out)
    
    model = tf.keras.Model(inputs=[model_input],outputs=[out])
    model.compile(loss="sparse_categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
    return model





## Implementation 3: Transfer Learning

In [ ]:
def model_MobileNet():
    base_model = tf.keras.applications.MobileNetV2(weights = 'imagenet', 
                                                   input_shape=(96, 96, 3), 
                                                  include_top = False)
    base_model.trainable = False
    
    for layer in base_model.layers:        
        print("layer:", layer.name, ", trainable:",layer.trainable)
        
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.4)(x)
    output = Dense(10, activation="softmax")(x)
    model = keras.Model(inputs=base_model.input, outputs = output)
    model.compile(loss="sparse_categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
    
    return model


### Data augmentations Type1: preprocessing layers

In [ ]:

rescale_layer = tf.keras.Sequential([layers.experimental.preprocessing.Rescaling(1./255)])

## Example from https://www.tensorflow.org/tutorials/images/data_augmentation
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255)
])

data_augmentation = tf.keras.Sequential([
  # Contrast
  layers.experimental.preprocessing.RandomContrast(0.5),
  # Rotate
  layers.experimental.preprocessing.RandomRotation(factor = (0.2), seed=42),
  # Zoom
  layers.experimental.preprocessing.RandomZoom(0.1),
  # Scaling
  #layers.experimental.preprocessing.Rescaling(1./255),
  #layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1),
])

batch_size = 32
AUTOTUNE = -1

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets
  #ds = ds.map(lambda x, y: (resize_and_rescale(x), y))
  x = resize_and_rescale(x)  

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set
  if augment:
    #ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y))
    ds = data_augmentation(x, training=True)
  # Use buffered prefecting on all datasets
  return ds.prefetch(-1)

### Data augmentations Type2: ImageDataGenerator


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1/255.,
                             zoom_range = 0.3,
                             height_shift_range = 0.3,
                            width_shift_range = 0.3,
                            rotation_range = 25,
                            shear_range=0.5,
                            fill_mode="constant",
                            cval=0
                            )
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

效果不錯:

zoom_range = 0.2,height_shift_range = 0.2,width_shift_range = 0.2,rotation_range = 20,shear_range=0.4

zoom_range = 0.1,height_shift_range = 0.1,width_shift_range = 0.1,rotation_range = 10,shear_range=0.2


### Helper functions

In [ ]:
def summarize_diagnostics(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['training', 'validation'])
    plt.figure()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['loss', 'validation loss'])
    plt.figure()
    
    plt.show()

from keras.callbacks import LearningRateScheduler, EarlyStopping
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
earlystop = EarlyStopping(patience = 15)

# Version a: Implement in module
def do_module_CNN(x_train, y_train, x_valid, y_valid):
    model_cnn = module_CNN()
    history = model_cnn.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                                      steps_per_epoch= x_train.shape[0] // 64,
                                      epochs=100,verbose=2,
                                      validation_data=valid_datagen.flow(x_valid, y_valid, batch_size=64),
                                      validation_steps= x_valid.shape[0] // 64,
                                      callbacks=[annealer, earlystop])

    return model_cnn, history

#Version b: Implement in function call
def do_CNN(x_train, y_train, x_valid, y_valid):
    model_cnn = model_CNN()
    history = model_cnn.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                                      steps_per_epoch= x_train.shape[0] // 64,
                                      epochs=100,verbose=2,
                                      validation_data=valid_datagen.flow(x_valid, y_valid, batch_size=64),
                                      validation_steps= x_valid.shape[0] // 64,
                                      callbacks=[annealer, earlystop])

    return model_cnn, history

def do_MobileNet(x_train, y_train, x_valid, y_valid):
    model_mbn = model_MobileNet()
    history1 = model_mbn.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                                      steps_per_epoch= x_train.shape[0] // 64,
                                      epochs=100,verbose=2,
                                      validation_data=valid_datagen.flow(x_valid, y_valid, batch_size=64),
                                      validation_steps= x_valid.shape[0] // 64,
                                      callbacks=[annealer, earlystop])
    print("=========Phase 2: Unfreeze lower layers========")
    model_mbn.trainable=True
    for layer in model_mbn.layers:        
        print("layer:", layer.name, ", trainable:",layer.trainable)
    model_mbn.compile(loss="sparse_categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
    history2 = model_mbn.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                                      steps_per_epoch= x_train.shape[0] // 64,
                                      epochs=100,verbose=2,
                                      validation_data=valid_datagen.flow(x_valid, y_valid, batch_size=64),
                                      validation_steps= x_valid.shape[0] // 64,
                                      callbacks=[annealer, earlystop])    
    return model_mbn, history2

## Run model

In [ ]:
import time
start_time = time.time()
model, history = do_MobileNet(x_train, y_train, x_valid, y_valid)
print("--- Model Train: %s seconds ---" % (time.time() - start_time))

summarize_diagnostics(history)

## End: Predict Test set 

In [ ]:
start_time = time.time()

#y_hat = model.predict(x_test, batch_size=64)
y_hat = model.predict_generator(test_datagen.flow(x_test, shuffle=False))
y_pred = np.argmax(y_hat,axis=1)

print("--- Model Predict: %s seconds ---" % (time.time() - start_time))
print("y shape", y_pred.shape)

In [ ]:
pd_submit=pd.DataFrame({"id": list(range(len(y_pred))),"label": y_pred})
pd_submit.to_csv('submission.csv',index=False,header=True)

In [ ]:
pd_submit['label'].value_counts()